## Исследование надежности заемщиков.

Проведения исследования для проверки гипотез как различные переменные влияют на погашения заемщика кредита в срок.
Результаты исследования будут учтены для построения модели *кредитного скролинга* - специальной системы, которая будет оценивать способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** — проверьте гипотезы:
 * *Влияет ли семейное положение заёмщика на погошения им кредита в срок?* - запрос заказчика
 * *Влияет ли количество детей заёмщика на возврат кредита в срок?* - запрос заказчика
 * Влияет ли уровень дохода и возвратом кредита в срок?
 * Как разные цели кредита влияют на его возврат в срок?
 
**Входные данный** - статистика о платёжеспособности клиентов предоставленные банком заказчиком. `data.csv`

**Ход исследования**
1. **Обзор данных.** Мы проведем загрузку и обзор предоставленных данных. 
2. **Предобработка данных.** Проверим данные на ошибки и пропуски, поищем возможность исправить самые критические ошибки. Оценив их влияние на исследования.
3. **Проверка гипотез.** Подтвердим или опровергнем выдвенутые гипотезы.
4. **Результаты и выводы** Зделаем заключение основываясь на результатах.

### Шаг 1. Обзор данных

Для загрузки и обработки данных мы воспльзуемся библеотекой pandas

In [1]:
import pandas as pd

Загрузим файл `data.csv` из папки `/datasets` и сохраним в переменной `dt`

In [2]:
try:
    df = pd.read_csv('/datasets/data.csv')
    display('Загрузка онлайн')
except:
    df = pd.read_csv('data.csv')
    display('Загрузка офлайн')

'Загрузка офлайн'

Выведем на экран первые 10 строк таблицы:

In [3]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Посмотрим общую информацию о таблице:

In [4]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Мы видим в таблице 12 столбцов, с различными типами данных.

Согласно документации к данным:
 - `children` — **количество детей в семье** тип данных `int64` целочисленное число
 - `days_employed` — **общий трудовой стаж в днях** тип данных `float64` вещественное число
 - `dob_years` — **возраст клиента в годах** тип данных `int64` целочисленное число
 - `education` — **уровень образования клиента** тип данных `object` текст
 - `education_id` — **идентификатор уровня образования** тип данных `int64` целочисленное число
 - `family_status` — **семейное положение** тип данных `object` текст
 - `family_status_id` — **идентификатор семейного положения** тип данных `int64` целочисленное число
 - `gender` — **пол клиента** тип данных `object` текст
 - `income_type` — **тип занятости** тип данных `object` текст
 - `debt` — **имел ли задолженность по возврату кредитов** тип данных `int64` целочисленное число
 - `total_income` — **ежемесячный доход** тип данных `float64` вещественное число
 - `purpose` — **цель получения кредита** тип данных `object` текст

В названиях колонок нарушения стиля нет.

Так же стоит обратить внимание на аномалию отрицательных значений в столбце общего трудового стажа.

И в колонке с образованиям применение разного регистра, при поиске и удалении дубликатов нужно все превести к нижнему регистру.

Количество значений в столбцах различается.
Проверим все столбцы на пропуски:

In [5]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В данных есть пропущеные значения, в столбцах `days_employed` и `total_income`.

In [6]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Шаг 2.1 Заполнение пропусков

В столбцах  "общий трудовой стаж в днях" `days_employed` и "ежемесячный доход" `total_income` есть пустые значения.

Посмотрим как выглядят пропуски на примере столбца "общий трудовой стаж в днях" `days_employed`, выведем первые 5 строк:

In [7]:
df.loc[df['days_employed'].isna()].head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Видим что пропуски в двух столбцах совпадают, возможно ошибка при выгрузке данных или технический сбой.
Совпадение в двух столбцах врят ли вызвано человеческий фактором, так что исключаем его.

Посчитаем какую долю долю составляют пропущенные значения:

In [8]:
display('Доля пустых значений в days_employed:{:.2%}'.format(df['days_employed'].isna().sum() / df.shape[0]))
display('Доля пустых значений в total_income:{:.2%}'.format(df['total_income'].isna().sum() / df.shape[0]))

'Доля пустых значений в days_employed:10.10%'

'Доля пустых значений в total_income:10.10%'

10% большая доля выборки.

Оптимальным решение в количественных переменных поставить медиальное значение, так как это центральное числовое значение набора данных, а не просто это среднее значение что может существеннее "исказить" результаты и выводы.

Так как в столбце со стажем аномальные данные, пока займемся столбцом `total_income`.

Выясним в каких категории занятости есть пропуски:

In [9]:
df.loc[df['total_income'].isna()]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

заполняем пустые значения медиальным

In [10]:
list_income_type = df['income_type'].unique() #список категорий

for item in list_income_type:
    median_income = df.loc[df['income_type'] == item,'total_income'].median() #получаем медиальное значение каждой категории
    df.loc[(df['income_type'] == item) & (df['total_income'] .isna()),'total_income'] = median_income #вставляем медиальное значение в пустые ячейки категрии

Проверим на примере категории гослужащих поменялись ли значения выведем первые 5 строк:

In [11]:
df.loc[df['income_type'] == 'госслужащий'].head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,150447.935283,образование
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,150447.935283,сделка с подержанным автомобилем
47,1,-2689.137274,33,высшее,0,гражданский брак,1,M,госслужащий,0,356277.909345,на проведение свадьбы
62,0,-7845.649233,48,Высшее,0,Не женат / не замужем,4,M,госслужащий,0,435388.195272,получение высшего образования
70,1,-2802.226671,28,ВЫСШЕЕ,0,женат / замужем,0,M,госслужащий,0,207561.466998,покупка коммерческой недвижимости


Видим что при пропущеном значении в столбце `days_employed` столбец `total_income` заполнен медиальном значением

Подсчитаем количество пропусков после заполнения столбца `total_income`:

In [12]:
df['total_income'].isna().sum()

0

### Шаг 2.2 Проверка данных на аномалии и исправления.

Возвращаемся к столбцу `days_employed`, возможно при заполнение формы заявки перепутали ячейки с датами местами и вычисляя стаж он стал отрицательным, но так как есть и положительные значение пока трудно сакзать точно.

In [13]:
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Наверное нужно обратиться к заказчику и выяснить как вычислялось этот столбец, какие данные использовались чтобы избежать таких аномалий!

Пока выясним какой процент отрицательных значений а какой положительный:

In [14]:
display('Доля отрицательных значений в total_income:{:.2%}'.format(df.loc[df['days_employed'] < 0]['days_employed'].count() / df['days_employed'].count()))
display('Доля положительных значений в total_income:{:.2%}'.format(df.loc[df['days_employed'] > 0]['days_employed'].count() / df['days_employed'].count()))

'Доля отрицательных значений в total_income:82.20%'

'Доля положительных значений в total_income:17.80%'

Отрицательных значений 82 процента так что будем его брать как основу.
Посмотрим на минимальные и максимальные значения в этих диапозонов:

In [15]:
display('Минимальное отрицательное: {:.6}'.format(df.loc[df['days_employed'] < 0]['days_employed'].min()))
display('Максимальное отрицательное: {:.6}'.format(df.loc[df['days_employed'] < 0]['days_employed'].max()))
display('Минимальное положительное: {:.8}'.format(df.loc[df['days_employed'] > 0]['days_employed'].min()))
display('Максимальное положительное: {:.8}'.format(df.loc[df['days_employed'] > 0]['days_employed'].max()))

'Минимальное отрицательное: -18388.9'

'Максимальное отрицательное: -24.1416'

'Минимальное положительное: 328728.72'

'Максимальное положительное: 401755.4'

Слишком большой отрыв у положительных значений данные начинаются с 328728 тогда как отрицательные -24. Видемо что то с еденицами измерения возможно в отрицательных данных перепутаны колонки и вычисляя количество дней получаем отрицательное значение. Тогда как в положительной выборке даты не перепутанны но еденицы измерения не дни, а возможно часы. прикинем так ли это:

In [16]:
display(
    'Максимальный стаж (отрицательная выборка): {:.4} лет'
    .format(df.loc[df['days_employed'] < 0]['days_employed'].min()/365*-1)
)
display(
    'Максимальный стаж (положительная выборка): {:.6} лет'
    .format(df.loc[df['days_employed'] > 0]['days_employed'].max()/365)
)

'Максимальный стаж (отрицательная выборка): 50.38 лет'

'Максимальный стаж (положительная выборка): 1100.7 лет'

Видим что отрицательная выборка действительно измеряется днями тогда как положительная явно превышена, если только это не египетские боги которые живут несколько тысяч лет. Так что поделим на 24, предположив что данные предоставленны в часах.

In [17]:
display(
    'Максимальный стаж (положительная выборка): {:.6} лет'
    .format(df.loc[df['days_employed'] > 0]['days_employed'].max()/24/365)
)

'Максимальный стаж (положительная выборка): 45.8625 лет'

Больше похоже на истину. Приведем все данные к общему знаменателю. У положительной выборки данные разделим на 24, а у отрицательных убирем минус:

In [18]:
df.loc[df['days_employed'] > 0,'days_employed'] = df.loc[df['days_employed'] > 0,'days_employed']/24
df['days_employed'] = df['days_employed'].abs()

display('Минимальное: {:.6}'.format(df['days_employed'].min()))
display('Максимальное: {:.6}'.format(df['days_employed'].max()))

'Минимальное: 24.1416'

'Максимальное: 18388.9'

Поставим медиальное значение в пропущенные ячейки и посчитаем есть ли пустые яцейки в таблице:

In [19]:
df.loc[df['days_employed'] .isna(),'days_employed'] = df['days_employed'].median()
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Все пустых ячеек не осталось. Так же при изучения данных можно обратить внимание что в поле возраста есть нулевые значения, выведем данные:

In [20]:
df.loc[df['dob_years'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,14439.234121,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,16577.356876,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,2194.220567,0,среднее,1,женат / замужем,0,F,сотрудник,0,142594.396847,жилье
20462,0,14113.952856,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,покупка своего жилья
20577,0,13822.552977,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,недвижимость
21179,2,108.967042,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,строительство жилой недвижимости


101 ячейка это 0,4 % от всех данных. Можно удалить или оставить так как есть.

Отлично, еще при проверке обнаружилась еще одна аномалия, в столбце с количеством детей есть отрицательное значение,  Так же значение в 20 детей у 76 анкет наводит на мыль в опечатке или сбое(так как нет разброса после 5 сразу 20) или у банка отдельная программа для семей с 20 детьми.

In [21]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Избавимся от этой аномалий и повторно выведем данные:

In [22]:
df['children'] = df['children'].abs()
df.loc[df['children'] == 20, 'children'] = 2
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Все переходим к следущему шагу.

### Шаг 2.3. Изменение типов данных.

###### Преведем столбцы из вещесвенного типа в целочисленное для удобства восприятия:

In [23]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

###### Проверим данные:

In [24]:
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


Все заменилось, переходим к следущему шагу:

### Шаг 2.4. Удаление дубликатов.

Удалим явные дубликаты в нашем DataFrame.

И выведем для проверки общую информацию:

In [25]:
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21471 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21471 non-null  int64 
 1   days_employed     21471 non-null  int32 
 2   dob_years         21471 non-null  int64 
 3   education         21471 non-null  object
 4   education_id      21471 non-null  int64 
 5   family_status     21471 non-null  object
 6   family_status_id  21471 non-null  int64 
 7   gender            21471 non-null  object
 8   income_type       21471 non-null  object
 9   debt              21471 non-null  int64 
 10  total_income      21471 non-null  int32 
 11  purpose           21471 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 2.0+ MB


Из 21525 строк 54 были явными дубликатами.

Теперь колонку `education` проверим на уникальные значения чтобы произвести поиск неявных дубликатов:

In [26]:
df['education'].value_counts()

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Дубликаты могли появиться при внесении данных человеком, появились разные варинты написания.

Приведем все к нижнему регистру и опять выведем уникальные значения стобца:

In [27]:
df['education'] = df['education'].str.lower()
df['education'].value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Из 15 стало всего 5 уникальных значений.
Повторно удалим дубликаты и выведем общую информацию по таблице:

In [28]:
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21454 non-null  int64 
 1   days_employed     21454 non-null  int32 
 2   dob_years         21454 non-null  int64 
 3   education         21454 non-null  object
 4   education_id      21454 non-null  int64 
 5   family_status     21454 non-null  object
 6   family_status_id  21454 non-null  int64 
 7   gender            21454 non-null  object
 8   income_type       21454 non-null  object
 9   debt              21454 non-null  int64 
 10  total_income      21454 non-null  int32 
 11  purpose           21454 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 2.0+ MB


Еще 17 дубликатов удалены из таблицы.
Также при проверке колонки `family_status` на уникальные значения Есть едиственная категория с заглавной буквой :

In [29]:
df['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

Приведем все к нижнему регистру для общего стиля и выведим для проверки:

In [30]:
df['family_status'] = df['family_status'].str.lower()
df['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два новых датафрейма для визуального удобства и ускорение времени обработки данных.

Сделаем словарь для категорий по образованию выведем его отсортировав:

In [31]:
education_dict = df[['education','education_id']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict.sort_values('education_id')

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


Второй словарь по категориям семейного статуса:

In [32]:
family_dict = df[['family_status','family_status_id']]
family_dict = family_dict.drop_duplicates().reset_index(drop=True)
family_dict.sort_values('family_status_id')



,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


Удалим из исходного датафрейма столбцы `education`, `family_status`  оставив только столбцы с ID, чтобы к словарям обращаться по индификатору. Выведем первые 5 строк.

In [33]:
df = df.drop(columns=['education','family_status'])
df.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


Переходим к категоризации.

### Шаг 2.6. Категоризация дохода.

Создадим функцию `income_group` для групированию по уровню дохода.

Сделаем 5 категорий

 - **A** - с диапозоном дохода 1000001 и больше
 - **B** - с диапозоном дохода 200001–1000000
 - **C** - с диапозоном дохода 50001–200000
 - **D** - с диапозоном дохода 30001–50000
 - **E** - с диапозоном дохода 0–30000

In [34]:
def income_group(row):
    
    income = row['total_income']
    
    try:
        if income < 30001:
            return 'E'
        if 30000 < income < 50001:
            return 'D'
        if 50000 < income < 200001:
            return 'C'
        if 200000 < income < 1000001:
            return 'B'
        return 'A'
    except:
        display('Ошибка проверить тип данных в total_income')

#для проверки сделаем серию pandas и запустим нашу функцию
row_values = [25000]
row_columns = ['total_income']

row = pd.Series(data=row_values, index=row_columns)

income_group(row)

'E'

Применим нашу функцию на датафрейм и выведем 5 строк.

In [35]:
df['total_income_category'] = df.apply(income_group, axis=1)
df.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,14177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


Так же для вывода подготовим для словарь по уровню доходов:

In [36]:
income_values = [
    ['A','A от 1 000 001₽ и больше'],
    ['B','B от 200 001₽ - 1 000 000₽'],
    ['C','C от 50 001₽ - 200 000₽'],
    ['D','D от 30 001₽ - 50 000₽'],
    ['E','E от 0₽ - 30 000₽'],
]

income_columns = ['total_income_category','income_range']
income_group = pd.DataFrame(data=income_values , columns=income_columns)

income_group

,total_income_category,income_range
0,A,A от 1 000 001₽ и больше
1,B,B от 200 001₽ - 1 000 000₽
2,C,C от 50 001₽ - 200 000₽
3,D,D от 30 001₽ - 50 000₽
4,E,E от 0₽ - 30 000₽


### Шаг 2.7. Категоризация целей кредита.

Создадим функцию `purpose_group` для групированию по цели кредита.

Сделаем 4 категорий:

 - **операции с автомобилем**
 - **операции с недвижимостью**
 - **проведение свадьбы**
 - **получение образования**
 
 Посмотрим какие категорие есть в колонке `purpose`:

In [37]:
 df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Напишем функцию и проверим ее на проверочной строке:

In [38]:
def purpose_group(row):
    
    purpose = row['purpose']#берем ячейки из колонки purpose разбивая их в списки
    try:
        if 'авто' in purpose:
            return 'операции с автомобилем'
        elif 'недвиж' in purpose or 'жиль' in purpose:
            return 'операции с недвижимостью'
        elif 'свадьб' in purpose:
            return 'проведение свадьбы'
        elif 'образо' in purpose:
            return 'получение образования'
    except:
        display('Ошибка проверь слова тригеры в списке')

            
#сделаем проверочную Series            
row_values = ['получение дополнительного образования']
row_columns = ['purpose']

row = pd.Series(data=row_values, index=row_columns)

purpose_group(row)#запустим нашу функцию по проверочной переменной row

'получение образования'

Запустим функцию пременив ее на нашем DataFrame и затем выведем первые 5 строк для проверки:

In [39]:
df['purpose_category'] = df.apply(purpose_group, axis=1)
df.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,14177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


### Шаг 2.8. Дополнительная категоризация.

Так же с учетом правок по аномалиям проведем дополнительную класификацию по возрасту и стажу.

Напишем функцию `experience_group` по категоризации стажа:

 - **E** - до года
 - **D** - 1 - 3 лет
 - **C** - 3 - 8 лет
 - **B** - 8 - 20 лет
 - **A** - больше 20 лет


In [40]:
def experience_group(row):
    
    experience = row['days_employed']
    year = 365
    try:
        if experience < year + 1: 
            return 'E'
        if year < experience < 3 * year + 1:
            return 'D'
        if 3 * year < experience < 8 * year + 1:
            return 'C'
        if 8 * year < experience < 20 * year:
            return 'B'
        return 'A'
    except:
        display('Ошибка проверить тип данных в days_employed')

#для проверки сделаем серию pandas и запустим нашу функцию
row_values = [10 * 365]
row_columns = ['days_employed']

row = pd.Series(data=row_values, index=row_columns)

experience_group(row)

'B'

Так же для вывода подготовим для словарь по стажу:

In [41]:
experience_values = [
    ['A','A больше 20 лет'],
    ['B','B от 8 - 20 лет'],
    ['C','C от 3 - 8 лет'],
    ['D','D от 1 - 3 лет'],
    ['E','E до 1 года'],
]

experience_columns = ['experience_category','experience_range']
experience_groups = pd.DataFrame(data=experience_values , columns=experience_columns)

experience_groups

,experience_category,experience_range
0,A,A больше 20 лет
1,B,B от 8 - 20 лет
2,C,C от 3 - 8 лет
3,D,D от 1 - 3 лет
4,E,E до 1 года


In [42]:
df['experience_category'] = df.apply(experience_group, axis=1)
df.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,experience_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,A
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,B
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,B
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,B
4,0,14177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,A


Напишем функцию `age_group` по категоризации возроста:

 - **A** - до 30 лет
 - **B** - от 30 - 40 лет
 - **C** - от 40 - 60 лет
 - **D** - больше 60 лет

In [43]:
def age_group(row):
    
    experience = row['dob_years']
    
    try:
        if experience < 31: #
            return 'A'
        if 30 < experience < 41:
            return 'B'
        if 40 < experience < 61:
            return 'C'
        if 60 < experience < 150:
            return 'D'
        return display('Проверить возраст! или готовить книгу рекордов гинноса по долгожителям')
    except:
        display('Ошибка проверить тип данных в dob_years')

#для проверки сделаем серию pandas и запустим нашу функцию
row_values = [50]
row_columns = ['dob_years']

row = pd.Series(data=row_values, index=row_columns)

age_group(row)

'C'

In [44]:
df['age_category'] = df.apply(age_group, axis=1)
df.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,experience_category,age_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,A,C
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,B,B
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,B,B
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,B,B
4,0,14177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,A,C


Так же для вывода подготовим для словарь по возрасту:

In [45]:
age_values = [
    ['A','A меньше 30 лет'],
    ['B','B от 30 - 40 лет'],
    ['C','C от 40 - 60 лет'],
    ['D','D больше 60 лет'],
]

age_columns = ['age_category','age_range']
age_groups = pd.DataFrame(data=age_values , columns=age_columns)

age_groups

,age_category,age_range
0,A,A меньше 30 лет
1,B,B от 30 - 40 лет
2,C,C от 40 - 60 лет
3,D,D больше 60 лет


### Ответы на вопросы.

Перед вопросом можно посмотреть средний % проблем с кредитом по всей таблице:

In [46]:
df['debt'].mean()

0.08115036822970076

Около **8%**

Выведем общую информацию о базе данных:

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   children               21454 non-null  int64 
 1   days_employed          21454 non-null  int32 
 2   dob_years              21454 non-null  int64 
 3   education_id           21454 non-null  int64 
 4   family_status_id       21454 non-null  int64 
 5   gender                 21454 non-null  object
 6   income_type            21454 non-null  object
 7   debt                   21454 non-null  int64 
 8   total_income           21454 non-null  int32 
 9   purpose                21454 non-null  object
 10  total_income_category  21454 non-null  object
 11  purpose_category       21454 non-null  object
 12  experience_category    21454 non-null  object
 13  age_category           21454 non-null  object
dtypes: int32(2), int64(5), object(7)
memory usage: 2.3+ MB


##### Влияет ли семейное положение заёмщика на погошения им кредита в срок?

In [48]:
family_status_group = df.merge(family_dict, on='family_status_id', how='left').groupby(['family_status']).agg({'debt':'mean'})
family_status_group

,debt
family_status,
в разводе,0.071130
вдовец / вдова,0.065693
гражданский брак,0.093471
женат / замужем,0.075452
не женат / не замужем,0.097509


##### Вывод 1:

Самый низкий показатель в **7%** и **7.5%** у категории `в разводе` и `женат/замужем` соответственно, далее в около **9,5%** у категорий `не женат / не замужем` и `гражданский брак`.

И самый низкий показатель проблем с кредитами в **6,5%**, у категории `вдовец / вдова`

##### Влияет ли количество детей заёмщика на возврат кредита в срок?

In [49]:
children_group = df.groupby(['children']).agg({'debt':'mean'})
children_group

,debt
children,
0,0.075438
1,0.091658
2,0.094925
3,0.081818
4,0.097561
5,0.000000


##### Вывод 2: 

Самый высокий показатель это кредитор с `4` детьми - около **9,7%**

С `1` или `2` детьми **9,1%** и **9,5%** имели проблемы с возвратом в срок.

С `3` близко к среднему значению **8%**, без детей около **7,5%**

Ну и лидер, у семей с `5` детьми **0%**, можно посмотреть сколько всего таких анкет:

In [50]:
df.loc[df['children'] == 5]['children'].count()

9

Всего 9 анкет из 21 тыс. очень маленькая выборка для твердой оценки, что лучшие потенциальные кредиторы должны иметь 5 детей.

##### Влияет ли уровень дохода и возвратом кредита в срок?

In [51]:
total_income_group = df.merge(income_group, on='total_income_category', how='left').groupby(['income_range']).agg({'debt':'mean'})
total_income_group

,debt
income_range,
A от 1 000 001₽ и больше,0.080000
B от 200 001₽ - 1 000 000₽,0.070607
C от 50 001₽ - 200 000₽,0.084920
D от 30 001₽ - 50 000₽,0.060000
E от 0₽ - 30 000₽,0.090909


##### Вывод 3:

###### Зависимости проблем с возвратами кредита от уровня дохода не наблюдается.

Cамый низкий показатель в **6%** у клиентов с `D` - с диапозоном дохода **30 001₽ – 50 000₽**

Самый высокий в **9%** у категории `E` - с диапозоном дохода **0₽ – 30 000₽**

В категориях `A`, `B`, `C` -  **8%**, **7%**, **8,5%** соответственно.

##### Как разные цели кредита влияют на его возврат в срок?

In [52]:
purpose_group = df.groupby(['purpose_category']).agg({'debt':'mean'})
purpose_group

,debt
purpose_category,
операции с автомобилем,0.093590
операции с недвижимостью,0.072334
получение образования,0.092200
проведение свадьбы,0.080034


##### Вывод 4:

###### Цель кредита так же не сильно влияет на его возврат.

Больше всего проблем у категорий кредита связанный с `автомобилем` и `образованием`, **9,3%** и **9,2%**.

**7,2%** у `операциий с недвижимостью` и **8%** на `проведение свадьбы`

##### Как стаж влияет на его возврат займа в срок? 
*(учитываем, что это очень примерный параметр, из за аномальных значений и преобразований)*

In [53]:
experiences_group = df.merge(experience_groups, on='experience_category', how='left').groupby(['experience_range']).agg({'debt':'mean'})
experiences_group

,debt
experience_range,
A больше 20 лет,0.050452
B от 8 - 20 лет,0.064707
C от 3 - 8 лет,0.083899
D от 1 - 3 лет,0.112821
E до 1 года,0.105664


##### Вывод 5:

В категории стаж, лучший показатель у кого его `больше 20 лет` всего **5%**

И худшие `до 1 года` и `от 1-3 лет` **10,5%** и **11,2%** соответсвенно.

##### Как возраст влияет на его возврат в кредита срок? 

In [54]:
ages_group = df.merge(age_groups, on='age_category', how='left').groupby(['age_range']).agg({'debt':'mean'})
ages_group

,debt
age_range,
A меньше 30 лет,0.107648
B от 30 - 40 лет,0.095255
C от 40 - 60 лет,0.069851
D больше 60 лет,0.047507


###### Вывод 6:

В возрастной группе так же как и группе со стажем. 
Лучший показатель у возрастной категории `больше 60 лет` всего **4%**
Средний у категории `от 40 - 60 лет` **7%**
Ну самый большой процент у группы возрастом `меньше 30 лет` **10,8%** и **9,5%** у группы от `30 - 40 лет`

## Общий вывод:

##### Мы проверили две гипотезы по требованию заказчика и 4 свои гипотезы и установили:

1) Семейное положение не сильно влияет на выплату кредита в срок!
 - Лучшая категория `вдовец / вдова` c **6,5%** и худшии `не женат/не замужем` и `гражданский брак` около **9%**
 
2) Количество детей так же не сильно сказывается на сроке выплаты!
 - Лучшая категория `без детей` c **7,5%** и худший `4 ребенка` около **9,7%**
 
3) Уровень дохода не влияет на выплату кредита в срок! 
 - Лучшая категория `30 001₽ – 50 000₽` c **6%** и худший `0 - 30 000₽` около **9%**
 
4) Цели кредита не очень сказываетются на сроке выплаты!
 - Лучшая категория `операциий с недвижимостью` c **7,2%** и худший `операции с автомобилем` около **9,3%**
 
5) Стаж влияет на выплату кредита в срок! 
 - Лучшая категория `больше 20 лет` c **5%** и худшии `до 1 года` и `от 1-3 лет` около **11%**
 
6) Возраст так же сказывается на сроке выплаты!
 - Лучшая категория `старше 60 лет` c **4,7%** и худший `младше 30 лет` около **10,8%**
 
 